In [13]:
import pandas as pd
import numpy as np

In [6]:
items = pd.read_csv('items.csv', sep='|')

In [8]:
items.columns

Index(['pid', 'manufacturer', 'group', 'content', 'unit', 'pharmForm',
       'genericProduct', 'salesIndex', 'category', 'campaignIndex', 'rrp'],
      dtype='object')

In [9]:
items['unit'].unique()

array(['ST', 'G', 'ML', 'P', 'KG', 'M', 'L', 'CM'], dtype=object)

### Goal

The plan is to unify units in the following way:
* ML and L -> ML
* KG and G -> G
* CM and M -> CM
* ST -> ST
* P -> P

We keep ST and P as is, but we still give them a new feature that indicates that the quantity reported is in ST and P units, respectively.

Two exceptions were found in items.csv:

`1. content == 'PAK' (only once)`

`2. content == 'L   125' (only once)`

I wrote the unification script to take care of these cases even if we don't care to correct the dataset.

### Content-unit unification

In [55]:
def standardize_qty(content, unit):
    # Handle cases where content is a product
    content = content.upper().strip()
    if 'X' in content:
        operands = [float(op) for op in content.split('X')]
        content = operands[0] * operands[1]
    elif content == 'PAK':  # Only one case in all items
        content = 0
    elif content == 'L   125':  # Only one case in all items
        content = 125
    else:
        content = float(content)
        
    unit = unit.upper().strip()
    if unit == 'ML':
        return (content, 'ML')
    elif unit == 'L':
        return (content*1000, 'ML')
    elif unit == 'G':
        return (content, 'G')
    elif unit == 'KG':
        return (content*1000, 'G')
    elif unit == 'CM':
        return (content, 'CM')
    elif unit == 'M':
        return (content*100, 'CM')
    elif unit == 'ST':
        return (content, 'ST')
    elif unit == 'P':
        return (content, 'P')
    else:
        raise ValueError('unknown unit type')
        

In [56]:
# Add columns of zeroes for new unified unit features
len_items = len(items)
items['unified_ML'] = pd.Series(np.zeros((len_items,), dtype=np.float16), index=items.index)
items['unified_G'] = pd.Series(np.zeros((len_items,), dtype=np.float16), index=items.index)
items['unified_CM'] = pd.Series(np.zeros((len_items,), dtype=np.float16), index=items.index)
items['unified_ST'] = pd.Series(np.zeros((len_items,), dtype=np.float16), index=items.index)
items['unified_P'] = pd.Series(np.zeros((len_items,), dtype=np.float16), index=items.index)

In [57]:
for idx, row in items.iterrows():
    content, std_unit = standardize_qty(row['content'], row['unit'])
    unified_feature = 'unified_'+std_unit  # Will be 'unified_ML' if unit == 'ML'
    items.loc[idx, unified_feature] = content

### Quick validation

Everything seems to be in order

In [62]:
items[items['unit'] == 'M'][['content', 'unit', 'unified_ML', 'unified_G', 'unified_CM', 'unified_ST', 'unified_P']]

,content,unit,unified_ML,unified_G,unified_CM,unified_ST,unified_P
2855,30,M,0.0,0.0,3000.0,0.0,0.0
5382,20,M,0.0,0.0,2000.0,0.0,0.0
5969,35,M,0.0,0.0,3500.0,0.0,0.0
6122,20,M,0.0,0.0,2000.0,0.0,0.0
6461,50,M,0.0,0.0,5000.0,0.0,0.0
7940,1X5,M,0.0,0.0,500.0,0.0,0.0
9598,50,M,0.0,0.0,5000.0,0.0,0.0
9638,40,M,0.0,0.0,4000.0,0.0,0.0
9639,50,M,0.0,0.0,5000.0,0.0,0.0
9640,50,M,0.0,0.0,5000.0,0.0,0.0


In [63]:
items[items['unit'] == 'CM'][['content', 'unit', 'unified_ML', 'unified_G', 'unified_CM', 'unified_ST', 'unified_P']]

,content,unit,unified_ML,unified_G,unified_CM,unified_ST,unified_P
13469,100X220,CM,0.0,0.0,22000.0,0.0,0.0


In [64]:
items[items['unit'] == 'KG'][['content', 'unit', 'unified_ML', 'unified_G', 'unified_CM', 'unified_ST', 'unified_P']]

,content,unit,unified_ML,unified_G,unified_CM,unified_ST,unified_P
1029,1,KG,0.0,1000.0,0.0,0.0,0.0
2125,1.5,KG,0.0,1500.0,0.0,0.0,0.0
13078,1,KG,0.0,1000.0,0.0,0.0,0.0
15488,1,KG,0.0,1000.0,0.0,0.0,0.0
15791,5,KG,0.0,5000.0,0.0,0.0,0.0
15792,1.5,KG,0.0,1500.0,0.0,0.0,0.0


In [65]:
items[items['unit'] == 'G'][['content', 'unit', 'unified_ML', 'unified_G', 'unified_CM', 'unified_ST', 'unified_P']]

,content,unit,unified_ML,unified_G,unified_CM,unified_ST,unified_P
2,10,G,0.0,10.00,0.0,0.0,0.0
5,10,G,0.0,10.00,0.0,0.0,0.0
8,10,G,0.0,10.00,0.0,0.0,0.0
45,125,G,0.0,125.00,0.0,0.0,0.0
55,14X20,G,0.0,280.00,0.0,0.0,0.0
57,46,G,0.0,46.00,0.0,0.0,0.0
58,72,G,0.0,72.00,0.0,0.0,0.0
59,72,G,0.0,72.00,0.0,0.0,0.0
60,72,G,0.0,72.00,0.0,0.0,0.0
61,50,G,0.0,50.00,0.0,0.0,0.0


### Save to file

In [81]:
items.to_csv('items_unified_units.csv', index=False)